In [ ]:
!pip install geopy==2.3.0

In [ ]:
import gradio as gr
import pandas as pd
import pickle
from datetime import datetime
from geopy.geocoders import Nominatim
from google.colab import drive

drive.mount('/content/drive')

parking_info_df = pd.read_csv('/content/drive/My Drive/NUS-ISS AIS Projects/Project 2/Data/HDBCarparkInformation.csv')

# Load KNN model
with open('knn_model_v2.pkl', 'rb') as f:
    knn_model = pickle.load(f)

def get_singapore_locations():
    geolocator = Nominatim(user_agent="my_geocoder")  # Replace with a descriptive user agent
    location = geolocator.geocode("Singapore")

    # Get the bounding box for Singapore
    south, north, west, east = location.raw['boundingbox']

    # Use the bounding box to query for places within Singapore
    places = geolocator.geocode(query="*", viewbox=[west, south, east, north], bounded=1)

    # Extract and return the location names
    location_names = [place.address for place in places if place]
    return location_names

locations = get_singapore_locations()  # Store the list of locations

parking_info_df = parking_info_df.rename(columns={'car_park_no': 'car_park_number'})
parking_info_df = parking_info_df.drop('gantry_height', axis=1, errors='ignore')

# Define a function to get the best parking location
def get_best_parking_location(input_location, input_time):
    # Convert the input time to the required format (if needed)
    # Process location and time as model inputs
    model_input = [input_location, input_time]  # Modify as necessary for model input format
    prediction = knn_model.predict([model_input])

    recommendations_details = []
    for parking_id in prediction:
        parking_details = parking_info_df[parking_info_df['carpark_number'] == parking_id].iloc[0].to_dict()  # Convert to dictionary
        details_string = f"Location: {parking_details['address']}, Type: {parking_details['car_park_type']}, Parking System: {parking_details['type_of_parking_system']},  Free Parking: {parking_details['free_parking']},  Night Parking: {parking_details['night_parking']},  Car Park Decks: {parking_details['car_park_decks']},  Basement Parking: {parking_details['car_park_basement']}"  # Format details
        recommendations_details.append(details_string)

    return ", ".join(recommendations_details)  # Convert list to comma-separated string

# Create Gradio interface
location_dropdown = gr.inputs.Dropdown(choices=locations, label="Select or Type Location", interactive=True)
time_input = gr.inputs.Textbox(label="Enter Time (HH:MM)", placeholder="e.g., 14:30")
output_text = gr.outputs.Textbox(label="Best Parking Location")

gr.Interface(
    fn=get_best_parking_location,
    inputs=[location_dropdown, time_input],
    outputs=output_text,
    title="Best Parking Location Finder",
    description="Select a location and enter a time to find the best parking option available.",
).launch()
